In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import os
from helpers.spot import *
from helpers.misc import *

In [2]:
# constants
ON_REPEAT = '37i9dQZF1Epfk0vNhDMCzw'
REPEAT_REWIND = '37i9dQZF1EpCchDHzfQcYR'
HIPHOP_NL = '0Fa3x3yr0duQJStzGpnH7w'

METAS = ['name', 'explicit', 'popularity']
FEATURES = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

PLAYLIST = REPEAT_REWIND

In [3]:
# seting up
client_id = os.environ.get('SPOT_CLIENT_ID')
secret = os.environ.get('SPOT_SECRET')

auth_manager = SpotifyClientCredentials(client_id, secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# getting track meta and features
ids = []
track_metas = []
track_features = []

playlist_name = sp.playlist(PLAYLIST, fields='name')['name']
playlist_tracks = get_playlist_tracks(sp, PLAYLIST)

ids = parse_ids(playlist_tracks)
track_metas = parse_metas(playlist_tracks, METAS)
track_features_mess = get_audio_features(sp, ids)
# need to flatten the track_features list
track_features_mess = [item for sublist in track_features_mess for item in sublist]
track_features = parse_features(track_features_mess, FEATURES)

In [8]:
# data parsing
dict_data = dict((key, []) for key in (METAS+FEATURES))

for meta, features in zip(track_metas, track_features):
    mf = meta + features
    for key, value in zip(dict_data.keys(), mf):
        dict_data[key].append(value)

data = pd.DataFrame.from_dict(dict_data)
num_data = data.iloc[:,2:]

# save data
data.to_csv(path_or_buf=f'data/{playlist_name}.csv')